In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [140]:
#importing all csv files
domarar = pd.read_csv('csv/blak-domarar.csv', sep=';', header=0)
einstaklingar = pd.read_csv('csv/blak-einstaklingar.csv', sep=';', header=0)
forsvarsmenn = pd.read_csv('csv/blak-forsvarsmenn.csv', sep=';', header=0)
lid = pd.read_csv('csv/blak-lid.csv', sep=';', header=0)
lidimoti = pd.read_csv('csv/blak-lidimoti.csv', sep=';', header=0)
lidsmenn = pd.read_csv('csv/blak-lidsmenn.csv', sep=';', header=0)
lidsstjorar = pd.read_csv('csv/blak-lidsstjorar.csv', sep=';', header=0)
thjalfarar = pd.read_csv('csv/blak-thjalfarar.csv', sep=';', header=0)
mot = pd.read_csv('csv/blak-mot.csv', sep=';', header=0)

In [141]:
# drop all SyndarLids with an ID (SyndarlidID)
# (the reason for not dropping using SyndarLid is because I don't trust that column to be inserted correctly with [0,1])
lid = lid[lid['SyndarlidID'].isna()]
# then dropping it
lid = lid.drop(columns=['SyndarLid', 'SyndarlidID'])

In [100]:
#print(help(einstaklingar))

In [144]:
#print(domarar)
#print(einstaklingar)
#print(forsvarsmenn)
print(lid.columns)
#print(lidimoti)
#print(lidsmenn)
#print(lidsstjorar)
#print(mot)
#print(thjalfarar)

Index(['LidID', 'Nafn', 'Radnumer', 'LidaTegundID', 'Timastimpill'], dtype='object')


In [142]:
#print(domarar.describe())
#print(einstaklingar.describe())
#print(forsvarsmenn.describe())
print(lid.describe())
#print(lidimoti.describe())
#print(lidsmenn.describe())
#print(lidsstjorar.describe())
#print(mot.describe())
#print(thjalfarar.describe())

             LidID  LidaTegundID
count  1023.000000   1023.000000
mean   2862.463343     30.417400
std    2041.233494      9.222238
min      18.000000     20.000000
25%    1031.500000     21.000000
50%    2508.000000     32.000000
75%    4425.500000     39.000000
max    7172.000000     51.000000


In [103]:
#domarar.dtypes
einstaklingar.dtypes
#forsvarsmenn.dtypes
#lid.dtypes
#lidimoti.dtypes
#lidsmenn.dtypes
#lidsstjorar.dtypes
#mot.dtypes
#thjalfarar.dtypes

EinstID            int64
Nafn              object
Fdagur            object
Kyn               object
FelagISI          object
Netfang           object
Heimilisfang1    float64
Heimilisfang2    float64
Heimilisfang3    float64
Simi1             object
Simi2             object
Simi3             object
Timastimpill      object
Haed             float64
dtype: object

In [145]:
#make it easier to copy data
#domarar
#einstaklingar
#forsvarsmenn
#lidimoti
#lidsmenn
#lidsstjorar
#mot
#thjalfarar

#lid_nafn_valuecount = lid['Nafn'].value_counts()
#print(lid_nafn_valuecount)
#lid_nafn_valuecount[lid_nafn_valuecount < 2]
#lid['LidID'].value_counts() 

print(lid.shape)


#lid_nafn_valuecount2 = lid['Nafn'].value_counts()
#lid_nafn_valuecount2


(1023, 5)
(1023, 5)


In [59]:
#FINAL STEP (run after everything is done):
#save as new csv inside csv/new
pd.DataFrame(domarar).to_csv("csv/new/blak-domarar.csv")
pd.DataFrame(einstaklingar).to_csv("csv/new/blak-einstaklingar.csv")
pd.DataFrame(forsvarsmenn).to_csv("csv/new/blak-forsvarsmenn.csv.csv")
pd.DataFrame(lid).to_csv("csv/new/blak-lid.csv")
pd.DataFrame(lidimoti).to_csv("csv/new/blak-lidimoti.csv")
pd.DataFrame(lidsmenn).to_csv("csv/new/blak-lidsmenn.csv")
pd.DataFrame(lidsstjorar).to_csv("csv/new/blak-lidsstjorar.csv")
pd.DataFrame(mot).to_csv("csv/new/blak-mot.csv")
pd.DataFrame(thjalfarar).to_csv("csv/new/blak-thjalfarar.csv")